In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
from PIL import ImageFont, ImageDraw, Image
import sys

#actions = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','space','backspace','clear']
actions = ['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ','ㅏ','ㅑ','ㅓ','ㅕ','ㅗ','ㅛ','ㅜ','ㅠ','ㅡ','ㅣ','ㅐ','ㅒ','ㅔ','ㅖ','ㅚ','ㅟ','ㅢ','space','backspace','clear','sum']

seq_length = 30
secs_for_action = 15

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = []

        ret, img = cap.read()
        img = cv2.flip(img, 1) # 좌우 반전

        img = np.full(shape=(512,512,3),fill_value=255,dtype=np.uint8)
        img = Image.fromarray(img) #img배열을 PIL이 처리가능하게 변환

        draw = ImageDraw.Draw(img)
        myfont = ImageFont.truetype("C:/Users/kelly/IdeaProjects/Project mid/font/MaruBuri-Bold.ttf",20)
        org=(10,30)# 글자 표시할 위치

        draw.text(org, f'Waiting for collecting {action.upper()} action...', font=myfont, fill=(0,0,0)) # 추측으론 fill은 글자색 0,0,0은 검은색
#        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        # cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=font, fontScale=1, color=(255, 255, 255), thickness=2)
        img = np.array(img) # numpy가 처리할 수 있도록 다시 변환
        cv2.imshow('img', img)
        cv2.waitKey(2000)

        start_time = time.time()

#        if img is None:
#            print('Image load failed')
#            sys.exit()

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                        
                        

                    check = joint[5][0] - joint[17][0]
                    
                    #2차 추가
                    if(check >=0):
                        check = 1;
                    else:
                        check = 0;
                        
                    point = joint[5][0] - joint[0][0]
                    
                    
                    comp_tip_1 = joint[[4,8,12,16], 0:2]
                    comp_tip_2 = joint[[8,12,16,20], 0:2]
                                       
                    tip_to_tip = comp_tip_1 - comp_tip_2
                    
                    tip_to_tip = tip_to_tip.flatten()
                    
                    tip_to_tip = tip_to_tip / np.linalg.norm(tip_to_tip)
                                       
                                        
                    zero = joint[[0,0,0,0,0], 1:2 ]
                    tip = joint[[4,8,12,16,20], 1:2]
                    
                    zero_to_tip = zero - tip
                    
                    zero_to_tip = zero_to_tip / np.linalg.norm(zero_to_tip)
                    
                    
                    
                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :2] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :2] # Child joint
                    v = v2 - v1 # [20, 2]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree

                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)
                    
                    #2차 추가
            
                    point_y = joint[5][1] - joint[0][1]
            
                    if(point_y >=0):
                        point_y = 1;
                    else:
                        point_y = 0;
            
                    thumb4 = joint[4][0] - joint[5][0]
                    thumb4_y = joint[4][1] - joint[5][1]
            
                    if(thumb4 >=0):
                        thumb4= 1;
                    else:
                        thumb4= 0;
                        
                    if(thumb4_y >=0):
                        thumb4_y= 1;
                    else:
                        thumb4_y= 0;
                
            
                    second8 = joint[8][:2] - joint[17][:2]
                    second8 = second8.flatten()
                    second8 = second8 / np.linalg.norm(second8)

                    d = np.append(v.flatten(), check)
                    d = np.append(d, point)

                    d = np.append(d,zero_to_tip)
                    d = np.append(d,tip_to_tip)
                    
                    #2차 추가 합치기
                    d = np.append(d, point_y )
                    d = np.append(d,thumb4)
                    d = np.append(d,thumb4_y)
                    d = np.append(d,second8)
                    
                    print(point_y,thumb4, thumb4_y,second8)
                    
                    d = np.concatenate([d, angle_label])

                    data.append(d)

                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

        # Create sequence data
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    break

1 0 0 [-0.47967958  0.87744373]
1 0 0 [-0.47073634  0.88227394]
1 0 0 [-0.47452707  0.88024091]
1 0 0 [-0.47745625  0.87865552]
1 0 0 [-0.48060445  0.87693749]
1 0 0 [-0.47882504  0.87791035]
1 0 0 [-0.47837745  0.87815432]
1 0 0 [-0.47513456  0.87991315]
1 0 0 [-0.47631288  0.87927586]
1 0 0 [-0.47680569  0.87900872]
1 0 0 [-0.47553243  0.87969819]
1 0 0 [-0.47910799  0.87775596]
1 0 0 [-0.47976107  0.87739918]
1 0 0 [-0.47770193  0.87852198]
1 0 0 [-0.47581159  0.87954723]
1 0 0 [-0.47495997  0.8800074 ]
1 0 0 [-0.48011667  0.87720464]
1 0 0 [-0.47811046  0.87829971]
1 0 0 [-0.48016742  0.87717687]
1 0 0 [-0.47739157  0.87869067]
1 0 0 [-0.47439931  0.88030977]
1 0 0 [-0.47895979  0.87783684]
1 0 0 [-0.47295462  0.88108679]
1 0 0 [-0.47127093  0.8819885 ]
1 0 0 [-0.46788654  0.88378854]
1 0 0 [-0.46519249  0.88520955]
1 0 0 [-0.46292063  0.88639973]
1 0 0 [-0.46225986  0.88674451]
1 0 0 [-0.45722808  0.88934947]
1 0 0 [-0.4526142   0.89170644]
1 0 0 [-0.45115706  0.89244457]
1 0 0 [-

1 0 0 [-0.42554748  0.9049361 ]
1 0 0 [-0.42769149  0.90392477]
1 0 0 [-0.42701094  0.90424646]
1 0 0 [-0.42574416  0.90484359]
1 0 0 [-0.42336671  0.9059584 ]
1 0 0 [-0.42578308  0.90482527]
1 0 0 [-0.42799044  0.90378326]
1 0 0 [-0.42810406  0.90372945]
1 0 0 [-0.42633213  0.9045667 ]
1 0 0 [-0.42750131  0.90401473]
1 0 0 [-0.43012859  0.90276763]
1 0 0 [-0.42765688  0.90394114]
1 0 0 [-0.42703542  0.9042349 ]
1 0 0 [-0.42846297  0.90355934]
1 0 0 [-0.42856178  0.90351248]
1 0 0 [-0.43335346  0.90122404]
1 0 0 [-0.42594088  0.904751  ]
1 0 0 [-0.4286409   0.90347495]
1 0 0 [-0.43050923  0.90258617]
1 0 0 [-0.42688421  0.90430629]
1 0 0 [-0.42832302  0.90362569]
1 0 0 [-0.4306056  0.9025402]
1 0 0 [-0.43190251  0.9019203 ]
1 0 0 [-0.42778154  0.90388216]
1 0 0 [-0.41936577  0.90781736]
1 0 0 [-0.41999787  0.90752509]
1 0 0 [-0.41762107  0.90862129]
1 0 0 [-0.4101161   0.91203333]
1 0 0 [-0.41459672  0.91000525]
1 0 0 [-0.41168131  0.91132788]
1 0 0 [-0.41248409  0.9109648 ]
1 0 0 [-0.

0 1 0 [-0.67361553 -0.73908194]
0 1 0 [-0.67589288 -0.73699987]
0 1 0 [-0.67623707 -0.73668408]
0 1 0 [-0.67147826 -0.74102425]
0 1 0 [-0.67616006 -0.73675476]
0 1 0 [-0.67461801 -0.73816701]
0 1 0 [-0.68432527 -0.72917688]
0 1 0 [-0.6844824  -0.72902939]
0 1 0 [-0.68473761 -0.72878968]
0 1 0 [-0.68538806 -0.72817801]
0 1 0 [-0.68081149 -0.73245868]
0 1 0 [-0.69080053 -0.72304539]
0 1 0 [-0.68480485 -0.72872651]
0 1 0 [-0.6822133  -0.73115321]
0 1 0 [-0.6812623 -0.7320394]
0 1 0 [-0.67903243 -0.73410827]
0 1 0 [-0.67605704 -0.73684929]
0 1 0 [-0.67666969 -0.73628671]
0 1 0 [-0.67612347 -0.73678834]
0 1 0 [-0.66824388 -0.74394228]
0 1 0 [-0.66933126 -0.74296411]
0 1 0 [-0.67541247 -0.73744017]
0 1 0 [-0.67917559 -0.73397583]
0 1 0 [-0.67646506 -0.73647473]
0 1 0 [-0.68137415 -0.73193529]
0 1 0 [-0.68341674 -0.73002846]
0 1 0 [-0.67534993 -0.73749744]
0 1 0 [-0.66949616 -0.74281552]
0 1 0 [-0.66935264 -0.74294484]
0 1 0 [-0.67346853 -0.7392159 ]
0 1 0 [-0.66663802 -0.74538161]
0 1 0 [-0.

0 1 0 [-0.69551146 -0.71851501]
0 1 0 [-0.69439998 -0.71958924]
0 1 0 [-0.69373564 -0.72022973]
0 1 0 [-0.69849745 -0.71561255]
0 1 0 [-0.68924275 -0.72453049]
0 1 0 [-0.69301983 -0.72091852]
0 1 0 [-0.69629302 -0.71775764]
0 1 0 [-0.69356581 -0.72039327]
0 1 0 [-0.69434739 -0.71963998]
0 1 0 [-0.69472871 -0.71927187]
0 1 0 [-0.68822799 -0.72549447]
0 1 0 [-0.69258102 -0.72134009]
0 1 0 [-0.69336971 -0.72058202]
0 1 0 [-0.69212573 -0.72177695]
0 1 0 [-0.69422302 -0.71975996]
0 1 0 [-0.68899032 -0.72477054]
0 1 0 [-0.69021627 -0.72360313]
0 1 0 [-0.68924742 -0.72452605]
0 1 0 [-0.68467234 -0.72885101]
0 1 0 [-0.68361859 -0.72983945]
0 1 0 [-0.67979045 -0.7334064 ]
0 1 0 [-0.6784854  -0.73461389]
0 1 0 [-0.67526466 -0.73757551]
0 1 0 [-0.67446055 -0.73831089]
0 1 0 [-0.67789882 -0.73515521]
0 1 0 [-0.67645191 -0.7364868 ]
0 1 0 [-0.67508993 -0.73773545]
0 1 0 [-0.67973045 -0.73346201]
0 1 0 [-0.67708384 -0.73590589]
0 1 0 [-0.67583495 -0.737053  ]
0 1 0 [-0.6778704  -0.73518142]
0 1 0 [-

0 0 1 [-0.78969679 -0.61349733]
0 0 1 [-0.78957715 -0.61365131]
0 0 1 [-0.7814728  -0.62393931]
0 0 1 [-0.7878961 -0.6158082]
0 0 1 [-0.78634832 -0.61778339]
0 0 1 [-0.79058489 -0.61235245]
0 0 1 [-0.78836466 -0.61520823]
0 0 1 [-0.78698964 -0.61696621]
0 0 1 [-0.78793744 -0.6157553 ]
0 0 1 [-0.77869614 -0.62740124]
0 0 1 [-0.77746464 -0.62892665]
0 0 1 [-0.77511029 -0.63182595]
0 0 1 [-0.77977977 -0.62605392]
0 0 1 [-0.76920975 -0.63899638]
0 0 1 [-0.77534873 -0.63153333]
0 0 1 [-0.76618877 -0.64261556]
0 0 1 [-0.7787657 -0.6273149]
0 0 1 [-0.77853213 -0.62760475]
0 0 1 [-0.77706421 -0.62942133]
0 0 1 [-0.77065237 -0.63725578]
0 0 1 [-0.77093966 -0.63690819]
0 0 1 [-0.77667271 -0.62990436]
0 0 1 [-0.77937717 -0.62655505]
0 0 1 [-0.77553399 -0.63130581]
0 0 1 [-0.77277311 -0.63468238]
0 0 1 [-0.77643353 -0.63019915]
0 0 1 [-0.77230409 -0.63525302]
0 0 1 [-0.76819351 -0.64021772]
0 0 1 [-0.76657621 -0.64215334]
0 0 1 [-0.77271746 -0.63475013]
0 0 1 [-0.77610119 -0.63060839]
0 0 1 [-0.77

...

0 0 0 [ 0.05025384 -0.99873648]
0 0 0 [ 0.08139342 -0.99668205]
0 0 0 [ 0.08862179 -0.99606535]
0 0 0 [ 0.0842688  -0.99644306]
0 0 0 [ 0.08673227 -0.99623166]
0 0 0 [ 0.08839029 -0.99608592]
0 0 0 [ 0.09517646 -0.99546042]
0 0 0 [ 0.07412282 -0.99724912]
0 0 0 [ 0.05449265 -0.99851417]
0 0 0 [ 0.03338482 -0.99944257]
0 0 0 [ 0.02567301 -0.99967039]
0 0 0 [ 0.00681045 -0.99997681]
0 0 0 [-0.01661228 -0.99986201]
0 0 0 [-0.02030219 -0.99979389]
0 0 0 [-0.01327607 -0.99991187]
0 0 0 [-0.00903291 -0.9999592 ]
0 0 0 [-0.00851227 -0.99996377]
0 0 0 [ 0.00230806 -0.99999734]
0 0 0 [ 0.00156348 -0.99999878]
0 0 0 [ 0.03023151 -0.99954292]
0 0 0 [ 0.05176757 -0.99865916]
0 0 0 [ 0.07280955 -0.99734586]
0 0 0 [ 0.0701992  -0.99753299]
0 0 0 [ 0.07352901 -0.99729308]
0 0 0 [ 0.07288397 -0.99734043]
0 0 0 [ 0.09140602 -0.99581371]
0 0 0 [ 0.08194768 -0.99663663]
0 0 0 [ 0.08572031 -0.99631924]
0 0 0 [ 0.07742066 -0.99699852]
0 0 0 [ 0.08097588 -0.99671606]
0 0 0 [ 0.08039131 -0.99676338]
0 0 0 [ 

0 0 0 [-0.04417975 -0.9990236 ]
0 0 0 [-0.0499963 -0.9987494]
0 0 0 [-0.05656239 -0.99839907]
0 0 0 [-0.06594656 -0.99782316]
0 0 0 [-0.06636649 -0.99779531]
0 0 0 [-0.0695666  -0.99757731]
0 0 0 [-0.06106232 -0.99813396]
0 0 0 [-0.04227463 -0.99910603]
0 0 0 [-0.04366808 -0.99904609]
0 0 0 [-0.02781725 -0.99961303]
0 0 0 [-0.02351527 -0.99972348]
0 0 0 [-0.01441512 -0.9998961 ]
0 0 0 [-0.01941423 -0.99981153]
0 0 0 [-0.03619885 -0.99934461]
0 0 0 [-0.04354672 -0.99905139]
0 0 0 [-0.06655157 -0.99778299]
0 0 0 [-0.07844432 -0.9969185 ]
0 0 0 [-0.09551993 -0.99542752]
0 0 0 [-0.13063281 -0.99143082]
0 0 0 [-0.13078568 -0.99141066]
0 0 0 [-0.13906361 -0.99028345]
0 0 0 [-0.13395836 -0.99098696]
0 0 0 [-0.12764231 -0.99182027]
0 0 0 [-0.12172698 -0.99256362]
0 0 0 [-0.11941361 -0.99284459]
0 0 0 [-0.11741167 -0.99308333]
0 0 0 [-0.1122292  -0.99368235]
0 0 0 [-0.10577138 -0.99439047]
0 0 0 [-0.10097849 -0.99488861]
0 0 0 [-0.08328857 -0.99652547]
0 0 0 [-0.07597883 -0.99710943]
0 0 0 [-0.

0 0 1 [-0.65356669 -0.75686893]
0 0 1 [-0.65659801 -0.75424071]
0 0 1 [-0.65424657 -0.75628131]
0 0 1 [-0.65680822 -0.75405766]
0 0 1 [-0.66314882 -0.74848757]
0 0 1 [-0.65957407 -0.75163957]
0 0 1 [-0.65317975 -0.75720289]
0 0 1 [-0.6570863  -0.75381536]
0 0 1 [-0.66492352 -0.74691144]
0 0 1 [-0.66347208 -0.74820104]
0 0 1 [-0.65853571 -0.75254948]
0 0 1 [-0.66129202 -0.75012856]
0 0 1 [-0.66346585 -0.74820657]
0 0 1 [-0.65521051 -0.75544635]
0 0 1 [-0.65714318 -0.75376577]
0 0 1 [-0.6596511  -0.75157197]
0 0 1 [-0.66020835 -0.75108251]
0 0 1 [-0.66143179 -0.75000532]
0 0 1 [-0.6627433  -0.74884666]
0 0 1 [-0.66615367 -0.74581452]
0 0 1 [-0.66474155 -0.74707341]
0 0 1 [-0.66590429 -0.74603718]
0 0 1 [-0.66816474 -0.74401336]
0 0 1 [-0.66661945 -0.74539823]
0 0 1 [-0.66784719 -0.74429842]
0 0 1 [-0.67287977 -0.73975186]
0 0 1 [-0.67289438 -0.73973857]
0 0 1 [-0.67417792 -0.73856898]
0 0 1 [-0.67535564 -0.73749221]
0 0 1 [-0.67497274 -0.73784267]
0 0 1 [-0.67422866 -0.73852266]
0 0 1 [-

0 0 1 [-0.70777171 -0.70644122]
0 0 1 [-0.7036814  -0.71051565]
0 0 1 [-0.69658409 -0.71747516]
0 0 1 [-0.69667929 -0.71738272]
0 0 1 [-0.70033276 -0.71381652]
0 0 1 [-0.69829277 -0.71581227]
0 0 1 [-0.70274127 -0.7114455 ]
0 0 1 [-0.70151954 -0.71265022]
0 0 1 [-0.69879668 -0.71532035]
0 0 1 [-0.69560219 -0.71842717]
0 0 1 [-0.69671486 -0.71734817]
0 0 1 [-0.69387832 -0.72009227]
0 0 1 [-0.69122209 -0.72264239]
0 0 1 [-0.6890089  -0.72475288]
0 0 1 [-0.68716184 -0.72650437]
0 0 1 [-0.68829718 -0.72542883]
0 0 1 [-0.6893313  -0.72444624]
0 0 1 [-0.69013287 -0.72368268]
0 0 1 [-0.68994677 -0.7238601 ]
0 0 1 [-0.69037751 -0.7234493 ]
0 0 1 [-0.68972769 -0.72406886]
0 0 1 [-0.68454831 -0.7289675 ]
0 0 1 [-0.68419211 -0.72930183]
0 0 1 [-0.67974372 -0.73344971]
0 0 1 [-0.67600767 -0.73689459]
clear (450, 76)
clear (420, 30, 76)
0 0 1 [-0.42263852 -0.90629834]
0 0 1 [-0.42080898 -0.90714927]
0 0 1 [-0.41622041 -0.90926375]
0 0 1 [-0.42351306 -0.90589   ]
0 0 1 [-0.42249092 -0.90636716]
0 0 

0 0 1 [-0.5075464  -0.86162442]
0 0 1 [-0.51377137 -0.85792714]
0 0 1 [-0.51466138 -0.85739353]
0 0 1 [-0.51542266 -0.8569361 ]
0 0 1 [-0.5166996  -0.85616676]
0 0 1 [-0.51468458 -0.8573796 ]
0 0 1 [-0.51104359 -0.8595548 ]
0 0 1 [-0.515706   -0.85676562]
0 0 1 [-0.517817   -0.85549141]
0 0 1 [-0.51636849 -0.8563665 ]
0 0 1 [-0.51792859 -0.85542386]
0 0 1 [-0.51730895 -0.85579872]
0 0 1 [-0.51454151 -0.85746547]
0 0 1 [-0.51634026 -0.85638352]
0 0 1 [-0.51751387 -0.85567482]
0 0 1 [-0.51511786 -0.85711936]
0 0 1 [-0.51471971 -0.85735851]
0 0 1 [-0.51492027 -0.85723808]
0 0 1 [-0.51735852 -0.85576876]
0 0 1 [-0.51242517 -0.85873188]
0 0 1 [-0.51998597 -0.8541748 ]
0 0 1 [-0.51121859 -0.85945073]
0 0 1 [-0.51475668 -0.85733632]
0 0 1 [-0.51522296 -0.85705619]
0 0 1 [-0.51637605 -0.85636194]
0 0 1 [-0.51744941 -0.8557138 ]
0 0 1 [-0.52093987 -0.85359338]
0 0 1 [-0.51605888 -0.85655311]
0 0 1 [-0.51320831 -0.85826408]
0 0 1 [-0.51519989 -0.85707005]
0 0 1 [-0.51362501 -0.85801477]
0 0 1 [-